##Práctica 5. Análisis de sentimientos, polaridad u opinión.

Montes de oca Campos David              

5BV1

Ingeniería en inteligencia artificial

Ultima modificación: 26/06/2024

##Analisis de datos.

In [1]:
# Importar librerías
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import pickle
import re
import string
from nltk.corpus import stopwords, opinion_lexicon
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pysentiment2 as ps
import swifter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout



c:\Users\davic\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Cargar el dataset
dataset = pd.read_csv('Reviews.csv')

#Indicar el número de filas y columnas del dataset
print("Dimensiones del dataset (número de filas, número de columnas):", dataset.shape)


# Función para describir las columnas
def describir_columnas(dataset):
    columnas = dataset.columns
    for col in columnas:
        print(f"\nColumna: {col}")
        print(f"Propósito: Describir el {col} del producto")
        tipo_dato = dataset[col].dtype
        print(f"Tipo de dato: {tipo_dato}")
        if tipo_dato == 'object':
            print("Formato: Cadena de caracteres (texto)")
            print(f"Ejemplo de valores: {dataset[col].unique()[:5]}")
            print(f"Número de valores únicos: {dataset[col].nunique()}")
        elif tipo_dato in ['int64', 'float64']:
            print("Formato: Número (entero o decimal)")
            print(f"Ejemplo de valores: {dataset[col].unique()[:5]}")
            print(f"Valor mínimo: {dataset[col].min()}, Valor máximo: {dataset[col].max()}")
        else:
            print("Formato: Otro (revisar valores específicos)")
            print(f"Ejemplo de valores: {dataset[col].unique()[:5]}")
            print(f"Número de valores únicos: {dataset[col].nunique()}")

# Describir cada columna del dataset
print("\nDescripción de cada dimensión (columna):")
describir_columnas(dataset)

# Imprimir primeras 5 filas del dataset
print("\nPrimeras 5 filas del dataset:")
print(dataset.head())



Dimensiones del dataset (número de filas, número de columnas): (568454, 10)

Descripción de cada dimensión (columna):

Columna: Id
Propósito: Describir el Id del producto
Tipo de dato: int64
Formato: Número (entero o decimal)
Ejemplo de valores: [1 2 3 4 5]
Valor mínimo: 1, Valor máximo: 568454

Columna: ProductId
Propósito: Describir el ProductId del producto
Tipo de dato: object
Formato: Cadena de caracteres (texto)
Ejemplo de valores: ['B001E4KFG0' 'B00813GRG4' 'B000LQOCH0' 'B000UA0QIQ' 'B006K2ZZ7K']
Número de valores únicos: 74258

Columna: UserId
Propósito: Describir el UserId del producto
Tipo de dato: object
Formato: Cadena de caracteres (texto)
Ejemplo de valores: ['A3SGXH7AUHU8GW' 'A1D87F6ZCVE5NK' 'ABXLMWJIXXAIN' 'A395BORC6FGVXV'
 'A1UQRSCLF8GW1T']
Número de valores únicos: 256059

Columna: ProfileName
Propósito: Describir el ProfileName del producto
Tipo de dato: object
Formato: Cadena de caracteres (texto)
Ejemplo de valores: ['delmartian' 'dll pa' 'Natalia Corres "Natalia C

#Preprocesamiento (Eliminación de columnas, Justificar dimensiones, etc.)

In [3]:
# Eliminar columnas no deseadas
columnas_eliminar = ['UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Time']
dataset = dataset.drop(columnas_eliminar, axis=1)

# Mostrar información del dataset después de eliminar columnas
print("\n\nDataset después de eliminar columnas no deseadas:")
print(dataset.info())

# Mostrar descripción del dataset después de eliminar columnas
print("\nDescripción del dataset después de eliminar columnas no deseadas:")
print(dataset.describe())

# Convertir la columna 'Score' a variables categóricas
def convertir_score(score):
    if score == 1 or score == 2:
        return 'Negativo'
    elif score == 3:
        return 'Neutral'
    elif score == 4 or score == 5:
        return 'Positivo'
    
dataset['Score'] = dataset['Score'].apply(convertir_score)

# Imprimir primeras 5 filas del dataset después de convertir 'Score' a categórico
print("\nPrimeras 5 filas del dataset después de convertir 'Score' a categórico:")
print(dataset.head())

#Validar el balance de clases
print("\nBalance de clases:")
print(dataset['Score'].value_counts())


# Balancear las clases 

# Separar clases
negativo = dataset[dataset['Score'] == 'Negativo']
neutral = dataset[dataset['Score'] == 'Neutral']
positivo = dataset[dataset['Score'] == 'Positivo']

# Encontrar la clase minoritaria
min_count = min(len(negativo), len(neutral), len(positivo))

# Submuestrear clases a la cantidad de la clase minoritaria
negativo_resampled = resample(negativo, replace=False, n_samples=min_count, random_state=42)
neutral_resampled = resample(neutral, replace=False, n_samples=min_count, random_state=42)
positivo_resampled = resample(positivo, replace=False, n_samples=min_count, random_state=42)

# Combinar clases balanceadas
dataset_balanced = pd.concat([negativo_resampled, neutral_resampled, positivo_resampled])

# Verificar nueva distribución de clases
print("\nDistribución de clases después de balancear:")
print(dataset_balanced['Score'].value_counts())

# Mostrar primeras 5 filas del dataset balanceado
print("\nPrimeras 5 filas del dataset balanceado:")
print(dataset_balanced.head())

# Guardar dataset balanceado para análisis posterior
dataset_balanced.to_csv('Reviews_balanced.csv', index=False)





Dataset después de eliminar columnas no deseadas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Id         568454 non-null  int64 
 1   ProductId  568454 non-null  object
 2   Score      568454 non-null  int64 
 3   Summary    568427 non-null  object
 4   Text       568454 non-null  object
dtypes: int64(2), object(3)
memory usage: 21.7+ MB
None

Descripción del dataset después de eliminar columnas no deseadas:
                  Id          Score
count  568454.000000  568454.000000
mean   284227.500000       4.183199
std    164098.679298       1.310436
min         1.000000       1.000000
25%    142114.250000       4.000000
50%    284227.500000       5.000000
75%    426340.750000       5.000000
max    568454.000000       5.000000

Primeras 5 filas del dataset después de convertir 'Score' a categórico:
   Id   ProductId     Score           

#4 y  5. Limpieza de datos y normalización de datos.

In [4]:
# Función de limpieza de texto
def limpiar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    
    # Eliminar números telefónicos, precios y direcciones (simplificado)
    texto = re.sub(r'\b\d{10,11}\b', '', texto)  # Números telefónicos
    texto = re.sub(r'\$\d+(\.\d{2})?', '', texto)  # Precios en formato $xxx.xx
    texto = re.sub(r'\d{1,2} [a-zA-Z]{3,9} [a-zA-Z]{3,9}', '', texto)  # Direcciones
    
    # Eliminar signos de puntuación
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    
    # Eliminar términos alfanuméricos y números
    texto = re.sub(r'\b\w*\d\w*\b', '', texto)
    
    # Tokenizar el texto
    palabras = word_tokenize(texto)
    
    # Eliminar stopwords
    stop_words = set(stopwords.words('english'))
    palabras_limpias = [word for word in palabras if word not in stop_words]
    
    # Unir las palabras limpias en una cadena
    texto_limpio = ' '.join(palabras_limpias)
    
    return texto_limpio

# Aplicar la función de limpieza a la columna 'Text'
dataset['Text'] = dataset['Text'].apply(limpiar_texto)

# Imprimir primeras 5 filas del dataset después de la limpieza de texto
print("\nPrimeras 5 filas del dataset después de la limpieza de texto:")
print(dataset.head())

#Vectorizar las reseñas, usando el método One-Hot Encoding
onehot_encoder = OneHotEncoder(sparse_output=False)

# Ajustar y transformar la columna 'Score' a One-Hot Encoding a los primeros 1000 registros
onehot_encoded = onehot_encoder.fit_transform(dataset['Score'][:1000].values.reshape(-1, 1))

# Imprimir las primeras 5 filas de la matriz One-Hot Encoding
print("\nMatriz One-Hot Encoding de las primeras 5 filas:")
print(onehot_encoded[:5])




Primeras 5 filas del dataset después de la limpieza de texto:
   Id   ProductId     Score                Summary  \
0   1  B001E4KFG0  Positivo  Good Quality Dog Food   
1   2  B00813GRG4  Negativo      Not as Advertised   
2   3  B000LQOCH0  Positivo  "Delight" says it all   
3   4  B000UA0QIQ  Negativo         Cough Medicine   
4   5  B006K2ZZ7K  Positivo            Great taffy   

                                                Text  
0  bought several vitality canned dog food produc...  
1  product arrived labeled jumbo salted peanutsth...  
2  confection around centuries light pillowy citr...  
3  looking secret ingredient robitussin believe f...  
4  great taffy great price wide assortment yummy ...  

Matriz One-Hot Encoding de las primeras 5 filas:
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


#6. Análisis de sentimientos con Harvard y Lexicon

In [5]:
# Crear objeto de análisis de sentimientos
hiv4 = ps.HIV4()

# Función para obtener el sentimiento de una reseña
def obtener_sentimiento_harvard(texto):
    tokens = word_tokenize(texto)
    sentimiento = hiv4.get_score(tokens)
    return sentimiento['Polarity']

# Aplicar la función de análisis de sentimientos a la columna 'Text'
dataset['Sentimiento_Harvard'] = dataset['Text'].apply(obtener_sentimiento_harvard)

# Imprimir primeras 50 filas del dataset después de análisis de sentimientos
print("\nPrimeras 50 filas del dataset después de análisis de sentimientos Harvard IV-4:")
print(dataset[['Text', 'Sentimiento_Harvard']].head(50))


Primeras 50 filas del dataset después de análisis de sentimientos Harvard IV-4:
                                                 Text  Sentimiento_Harvard
0   bought several vitality canned dog food produc...             1.000000
1   product arrived labeled jumbo salted peanutsth...             0.333333
2   confection around centuries light pillowy citr...             0.333333
3   looking secret ingredient robitussin believe f...            -0.999999
4   great taffy great price wide assortment yummy ...             1.000000
5   got wild hair taffy ordered five pound bag taf...            -0.333333
6   saltwater taffy great flavors soft chewy candy...             1.000000
7   taffy good soft chewy flavors amazing would de...             0.999999
8   right im mostly sprouting cats eat grass love ...             0.999999
9   healthy dog food good digestion also good smal...             0.000000
10  dont know cactus tequila unique combination in...             0.200000
11  one boys needed

In [6]:
# Función para obtener el sentimiento usando Opinion Lexicon
palabras_positivas = set(opinion_lexicon.positive())
palabras_negativas = set(opinion_lexicon.negative())

def obtener_sentimiento_opinion_lexicon(texto):
    tokens = word_tokenize(texto)
    sentimiento = 0
    for token in tokens:
        if token in palabras_positivas:
            sentimiento += 1
        elif token in palabras_negativas:
            sentimiento -= 1
    return sentimiento

# Aplicar la función de análisis de sentimientos a la columna 'Text' usando swifter para acelerar el proceso
dataset['Sentimiento_Opinion'] = dataset['Text'].swifter.apply(obtener_sentimiento_opinion_lexicon)

# Imprimir primeras 50 filas del dataset después de análisis de sentimientos
print("\nPrimeras 50 filas del dataset después de análisis de sentimientos Opinion Lexicon:")
print(dataset[['Text', 'Sentimiento_Opinion']].head(50))

Pandas Apply: 100%|██████████| 568454/568454 [01:05<00:00, 8726.38it/s] 


Primeras 50 filas del dataset después de análisis de sentimientos Opinion Lexicon:
                                                 Text  Sentimiento_Opinion
0   bought several vitality canned dog food produc...                    2
1   product arrived labeled jumbo salted peanutsth...                   -1
2   confection around centuries light pillowy citr...                    2
3   looking secret ingredient robitussin believe f...                    1
4   great taffy great price wide assortment yummy ...                    3
5   got wild hair taffy ordered five pound bag taf...                    1
6   saltwater taffy great flavors soft chewy candy...                    3
7   taffy good soft chewy flavors amazing would de...                    5
8   right im mostly sprouting cats eat grass love ...                    2
9   healthy dog food good digestion also good smal...                    3
10  dont know cactus tequila unique combination in...                    8
11  one boys nee

#Análisis de sentimientos con algoritmos de aprendizaje de máquina

In [7]:
# Vectorizar el texto usando TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500)
X = tfidf_vectorizer.fit_transform(dataset['Text'])
y = dataset['Score']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Regresión Logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Predecir las etiquetas de los datos de prueba
y_pred = logistic_model.predict(X_test)

# Imprimir el reporte de clasificación
print("\nReporte de clasificación del modelo de Regresión Logística:")
print(classification_report(y_test, y_pred))

C:\Users\davic\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Reporte de clasificación del modelo de Regresión Logística:
              precision    recall  f1-score   support

    Negativo       0.63      0.49      0.55     16181
     Neutral       0.45      0.08      0.14      8485
    Positivo       0.86      0.96      0.91     89025

    accuracy                           0.83    113691
   macro avg       0.65      0.51      0.53    113691
weighted avg       0.80      0.83      0.80    113691



In [8]:
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Árbol de Decisión con la muestra de datos
tree_model = DecisionTreeClassifier(max_depth=30, min_samples_split=10, min_samples_leaf=5)
tree_model.fit(X_train_sample, y_train_sample)

# Predecir las etiquetas de los datos de prueba
y_pred = tree_model.predict(X_test)

# Imprimir el reporte de clasificación
print("\nReporte de clasificación del modelo de Árbol de Decisión con datos reducidos:")
print(classification_report(y_test, y_pred))


Reporte de clasificación del modelo de Árbol de Decisión con datos reducidos:
              precision    recall  f1-score   support

    Negativo       0.49      0.31      0.38     16181
     Neutral       0.30      0.13      0.18      8485
    Positivo       0.83      0.93      0.88     89025

    accuracy                           0.79    113691
   macro avg       0.54      0.46      0.48    113691
weighted avg       0.75      0.79      0.76    113691



In [9]:
# Convertir etiquetas one-hot a 1D si es necesario
if len(y.shape) > 1 and y.shape[1] > 1:
    y = np.argmax(y.values, axis=1)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=0.2, random_state=42)  

# Inicializar el modelo de Máquinas de Soporte Vectorial
svm_model = SVC(kernel='linear', C=1, gamma='scale')


# Entrenar el modelo con todos los datos de entrenamiento
svm_model.fit(X_train_sample, y_train_sample)

# Predecir las etiquetas de los datos de prueba
y_pred = svm_model.predict(X_test)

# Imprimir el reporte de clasificación
print("\nReporte de clasificación del modelo de Máquinas de Soporte Vectorial:")
print(classification_report(y_test, y_pred))



Reporte de clasificación del modelo de Máquinas de Soporte Vectorial:


C:\Users\davic\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\davic\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    Negativo       0.63      0.46      0.53     16181
     Neutral       0.00      0.00      0.00      8485
    Positivo       0.85      0.97      0.91     89025

    accuracy                           0.83    113691
   macro avg       0.49      0.48      0.48    113691
weighted avg       0.75      0.83      0.78    113691



C:\Users\davic\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
# Cargar el dataset balanceado
dataset = pd.read_csv('Reviews_balanced.csv')

# seleccionar un subconjunto de ejemplos de cada clase
subset_size = 20000
balanced_dataset = pd.DataFrame()

# Iterar sobre cada clase y seleccionar subset_size ejemplos de cada una
for score in dataset['Score'].unique():
    subset = dataset[dataset['Score'] == score].head(subset_size)
    balanced_dataset = pd.concat([balanced_dataset, subset], ignore_index=True)

# Cargar X y y del dataset balanceado
X = balanced_dataset['Text']
y = balanced_dataset['Score']

# Aplicar TF-IDF Vectorizer
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)

# Reducir dimensionalidad con TruncatedSVD
svd = TruncatedSVD(n_components=1)  
X_reduced = svd.fit_transform(X_tfidf)

# Inicializar modelos
logistic_model = LogisticRegression(max_iter=1)
tree_model = DecisionTreeClassifier()
svm_model = SVC()

# Paralelización con cross_val_score usando n_jobs=-1
try:
    logistic_scores = cross_val_score(logistic_model, X_reduced, y, cv=2, n_jobs=-1)
    tree_scores = cross_val_score(tree_model, X_reduced, y, cv=2, n_jobs=-1)
    svm_scores = cross_val_score(svm_model, X_reduced, y, cv=2, n_jobs=-1)

    # Imprimir los puntajes de cross-validation
    print("\nPuntajes de cross-validation:")
    print("Regresión Logística:", logistic_scores)
    print("Árbol de Decisión:", tree_scores)
    print("Máquinas de Soporte Vectorial:", svm_scores)
except ValueError as e:
    print("Error:", e)



Puntajes de cross-validation:
Regresión Logística: [0.33336667 0.33336667]
Árbol de Decisión: [0.43883333 0.43663333]
Máquinas de Soporte Vectorial: [0.3806     0.38026667]


In [11]:

tokenizer = Tokenizer(num_words=5000, oov_token='')
tokenizer.fit_on_texts(dataset['Text'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(dataset['Text'])
padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post')

# Convertir etiquetas de sentimiento a one-hot encoding
labels = pd.get_dummies(dataset['Score'])

# Asegurarse de que las dimensiones coincidan
assert len(padded_sequences) == len(labels), "Las dimensiones de padded_sequences y labels no coinciden."

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


# Definir el modelo
model = Sequential()
model.add(Embedding(5000, 100, input_length=100))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
y_pred = np.argmax(model.predict(X_test), axis=-1)
accuracy = accuracy_score(np.argmax(y_test, axis=-1), y_pred)
print("Accuracy:", accuracy)

# Guardar el modelo entrenado
model.save('sentiment_analysis_model.h5')

# Guardar el tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)



c:\Users\davic\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 82s 6ms/step - accuracy: 0.8410 - loss: 0.4498 - val_accuracy: 0.8769 - val_loss: 0.3306
Epoch 2/15
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 81s 6ms/step - accuracy: 0.8853 - loss: 0.3126 - val_accuracy: 0.8910 - val_loss: 0.3053
Epoch 3/15
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 82s 6ms/step - accuracy: 0.9069 - loss: 0.2547 - val_accuracy: 0.8955 - val_loss: 0.3019
Epoch 4/15
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 76s 5ms/step - accuracy: 0.9229 - loss: 0.2103 - val_accuracy: 0.8975 - val_loss: 0.3061
Epoch 5/15
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 8874s 624ms/step - accuracy: 0.9345 - loss: 0.1774 - val_accuracy: 0.8977 - val_loss: 0.3395
Epoch 6/15
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 178s 12ms/step - accuracy: 0.9435 - loss: 0.1516 - val_accuracy: 0.8993 - val_loss: 0.3871
Epoch 7/15
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 201s 14ms/step - accuracy: 0.9516 - loss: 0.1299 - val_accuracy: 0.8986 - val_loss: 0.3904
Epoch 8/15
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 162s 11ms/step - 

Accuracy: 0.890088045667643


In [12]:
loaded_model = load_model('sentiment_analysis_model.h5')
with open('tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

# Definir una función para predecir el sentimiento del texto de entrada
def predict_sentiment(text):
    # Tokenizar y hacer padding del texto de entrada
    text_sequence = loaded_tokenizer.texts_to_sequences([text])
    text_sequence = pad_sequences(text_sequence, maxlen=100)

    # Hacer una predicción usando el modelo cargado
    predicted_sentiment = loaded_model.predict(text_sequence)
    sentiment_label = np.argmax(predicted_sentiment)
    
    # Mapear el índice de la etiqueta de sentimiento a su categoría correspondiente
    if sentiment_label == 0:
        return 'Negative'
    elif sentiment_label == 1:
        return 'Neutral'
    else:
        return 'Positive'

#Uso de la función predict_sentiment
# Pasar 10 reseñas de text del dataset para predecir el sentimiento
for i in range(10):
    text = dataset['Text'][i]
    sentiment = predict_sentiment(text)
    print(f"Texto: {text}")
    print(f"Sentimiento: {sentiment}")
    print()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Texto: bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better
Sentimiento: Positive

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Texto: product arrived labeled jumbo salted peanutsthe peanuts actually small sized unsalted sure error vendor intended represent product jumbo
Sentimiento: Negative

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Texto: confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story cs lewis lion witch wardrobe treat seduces edmund selling brother sisters witch
Sentimiento: Positive

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Texto: looking secret ingredient robitussin believe found got addition root beer extract ordered good made cherry soda flavor medicinal
Sentimiento: Negative

1/1 ━━━━━━

In [34]:
# Cargar el dataset balanceado
dataset = pd.read_csv('Reviews_balanced.csv')

# Convertir los documentos a secuencias de enteros
sequences = tokenizer.texts_to_sequences(dataset['Text'])
padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post')

# Convertir etiquetas de sentimiento a one-hot encoding
labels = pd.get_dummies(dataset['Score']).values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Cargar los embeddings preentrenados (GloVe)
embeddings_index = dict()
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Crear la matriz de embeddings
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Definir el modelo
model = Sequential()
embedding_layer = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=100, trainable=False)
model.add(embedding_layer)
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
y_pred = np.argmax(model.predict(X_test), axis=-1)
accuracy = accuracy_score(np.argmax(y_test, axis=-1), y_pred)
print("Accuracy:", accuracy)

# Guardar el modelo entrenado
model.save('sentiment_analysis_model.h5')

# Guardar el tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


loaded_model = load_model('sentiment_analysis_model.h5')
with open('tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

# Definir una función para predecir el sentimiento del texto de entrada
def predict_sentiment(text):
    # Tokenizar y hacer padding del texto de entrada
    text_sequence = loaded_tokenizer.texts_to_sequences([text])
    text_sequence = pad_sequences(text_sequence, maxlen=100)

    # Hacer una predicción usando el modelo cargado
    predicted_sentiment = loaded_model.predict(text_sequence)
    sentiment_label = np.argmax(predicted_sentiment)
    
    # Mapear el índice de la etiqueta de sentimiento a su categoría correspondiente
    if sentiment_label == 0:
        return 'Negative'
    elif sentiment_label == 1:
        return 'Neutral'
    else:
        return 'Positive'

# Ejemplo de uso
text_input = "I have an absolute passion for deep, dark hot chocolate. I want chocolate which tastes like *chocolate*, rich and flavorful, with a hint of a bite. You can savor chocolate like that."
predicted_sentiment = predict_sentiment(text_input)
print(predicted_sentiment)

NameError: name 'vocab_size' is not defined